# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto Submit (Part I)

#### I. Codes from Part I

In [12]:
import numpy as np 
import pandas as pd
import requests 
from bs4 import BeautifulSoup 

url = "https://www.wikizeroo.org/index.php?q=aHR0cHM6Ly9lbi53aWtpcGVkaWEub3JnL3dpa2kvTGlzdF9vZl9wb3N0YWxfY29kZXNfb2ZfQ2FuYWRhOl9N"
r = requests.get(url) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

postalcodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalcodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalcodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1

#### II. Adding Coordinates

In [13]:
import geocoder

colnames = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighbors = pd.DataFrame(columns=colnames)
coordinates = None


for data in range(0, len(postalcodes)-1):
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(postalcodes[data]))
    coordinates = g.latlng

    neighbors = neighbors.append({ 'PostalCode': postalcodes[data],
                                   'Borough': boroughs[data],
                                   'Neighborhood': neighborhoods[data],
                                   'Latitude': coordinates[0],
                                   'Longitude': coordinates[1]}, ignore_index=True)

neighbors.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752420,-79.329242
1,M4A,North York,Victoria Village,43.730600,-79.313265
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166
3,M6A,North York,Lawrence Heights,43.723270,-79.451286
4,M6A,North York,Lawrence Manor,43.723270,-79.451286
5,M7A,Queen's Park,Queen's Park,43.661150,-79.391715
6,M9A,Queen's Park,Queen's Park,43.662299,-79.528195
7,M1B,Scarborough,Rouge,43.811525,-79.195517
8,M1B,Scarborough,Malvern,43.811525,-79.195517
9,M3B,North York,Don Mills North,43.749055,-79.362227


#### III. Dimension of the Dataframe

In [14]:
neighbors.shape

(209, 5)